In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd

In [2]:
start_url = "https://www.fema.gov/disaster-reporter-data"
page_source = requests.get(start_url).text

In [3]:
soup = BeautifulSoup(page_source, 'lxml')

In [4]:
next_link = soup.findChild(attrs={'class':'next'}).find('a').attrs['href']
next_link

'/disaster-reporter-data?page=1'

In [5]:
header = ['Disaster Reports', 'Headlines', 'City', 'State',
          'Disaster Type', 'Photo Date', 'Report Date']

rows = soup.find('table').find('tbody').findAll('tr')

for row in rows:
    print(row.findAll('td'))

[<td><img alt="Photo: 1615 hwy 9" src="https://www.fema.gov/media-library-data/20180924-2005-19716-1696/disaster_reporter_20180924160539_search_preview.jpg"/><br/><a href="https://www.fema.gov/media-library-data/20180924-2005-19716-1696/disaster_reporter_20180924160539.jpg" title="View full size of - 1615 hwy 9">View Full Size</a><br/><a href="mailto:FEMA-New-Media@fema.dhs.gov?subject=Disaster%20Reporter:%20Review%20Image%20170920" title="Report this image - 1615 hwy 9">Report this Image</a></td>, <td>1615 hwy 9</td>, <td>Longs</td>, <td>SC</td>, <td>Floods</td>, <td>Sat, 09/22/2018 - 13:54</td>, <td class="active">Mon, 09/24/2018 - 16:05</td>]
[<td><img alt="Photo: Southport bc" src="https://www.fema.gov/media-library-data/20180924-1838-18167-5375/disaster_reporter_20180924143810_search_preview.jpg"/><br/><a href="https://www.fema.gov/media-library-data/20180924-1838-18167-5375/disaster_reporter_20180924143810.jpg" title="View full size of - Southport bc">View Full Size</a><br/><a hr

In [6]:
def parse_page(source):
    
    soup = BeautifulSoup(source, "lxml")
    
    ### Scrape the table
    rows = soup.find("table").find("tbody").findAll("tr")

    data = []

    for row in rows:

        data_row = []

        for value in row.findAll("td"):
            data_row.append(value.text)

        data.append(data_row)

    return data

def fetch_page(url):
    start_url = url
    return requests.get(start_url).text

In [8]:
start_url = "https://www.fema.gov/disaster-reporter-data"
page_source = fetch_page(start_url)

#is there a next link?
next_link = soup.findChild(attrs={'class':'next'}).find('a').attrs['href']
next_link

all_data = parse_page(page_source)

max_fetches = 50
current_page = 0

while True:
    
    if current_page >= max_fetches:
        break
    
    if next_link:
        print("Fetching next page...{}".format(current_page))
        next_url = 'https://www.fema.gov{}'.format(next_link)
        next_page_source = fetch_page(next_url)
        page_data = parse_page(next_page_source)
        all_data.extend(page_data)
    
    current_page += 1

#parse_page(page_source)

Fetching next page...0
Fetching next page...1
Fetching next page...2
Fetching next page...3
Fetching next page...4
Fetching next page...5
Fetching next page...6
Fetching next page...7
Fetching next page...8
Fetching next page...9
Fetching next page...10
Fetching next page...11
Fetching next page...12
Fetching next page...13
Fetching next page...14
Fetching next page...15
Fetching next page...16
Fetching next page...17
Fetching next page...18
Fetching next page...19
Fetching next page...20
Fetching next page...21
Fetching next page...22
Fetching next page...23
Fetching next page...24
Fetching next page...25
Fetching next page...26
Fetching next page...27
Fetching next page...28
Fetching next page...29
Fetching next page...30
Fetching next page...31
Fetching next page...32
Fetching next page...33
Fetching next page...34
Fetching next page...35
Fetching next page...36
Fetching next page...37
Fetching next page...38
Fetching next page...39
Fetching next page...40
Fetching next page...41
Fe

In [10]:
df = pd.DataFrame(all_data, columns = header)
df.head()

,Disaster Reports,Headlines,City,State,Disaster Type,Photo Date,Report Date
0,View Full SizeReport this Image,1615 hwy 9,Longs,SC,Floods,"Sat, 09/22/2018 - 13:54","Mon, 09/24/2018 - 16:05"
1,View Full SizeReport this Image,Southport bc,,,Hurricanes,"Wed, 09/19/2018 - 16:19","Mon, 09/24/2018 - 14:38"
2,View Full SizeReport this Image,Kathy Tripp's crushed barn,Newport,NC,Hurricanes,"Sat, 09/15/2018 - 05:14","Mon, 09/24/2018 - 12:17"
3,View Full SizeReport this Image,Conway sc,,,Hurricanes,"Sat, 09/15/2018 - 10:34","Sun, 09/23/2018 - 08:40"
4,View Full SizeReport this Image,My yard,,,Hurricanes,"Thu, 09/20/2018 - 04:53","Sat, 09/22/2018 - 17:38"


In [12]:
df.to_csv('fema.csv', encoding = 'UTF8')

In [13]:
df.shape

(1020, 7)